In [ ]:
!pip install __main__

ERROR: Invalid requirement: '__main__'


In [ ]:
import pandas as pd
from .preprocessors.paths import (PATH_TO_PLANT_A,
                                 PATH_TO_PLANT_B,
                                 PATH_TO_PLANT_C,
                                 PATH_TO_WEATHER)
from pv_diagnosis.preprocessors.plant_preprocessor import PlantPreprocessor
from pv_diagnosis.preprocessors.weather_preprocessor import WeatherPreprocessor

ModuleNotFoundError: No module named 'preprocessors'

In [ ]:

# read data power plants
data_power_plant_a = pd.read_csv("./data/data_power_plants/A.csv")
data_power_plant_b = pd.read_csv("./data/data_power_plants/B.csv")
data_power_plant_c = pd.read_csv("./data/data_power_plants/C.csv")

In [ ]:
class Preprocessing:
    def __init__(self, df):
        self.dataframe_raw = df
        dataframe_formatted_columns = self.format_columns(df)
        
        

    def format_columns(df):
        df.columns = df.columns.str.lower.str.replace("-","")

,Timestamp,Generation_kW,Grid_Feed-In_kW,Grid_Supply_kW,Overall_Consumption_Calc_kW
0,2019-01-01 00:00:00,0.000,0.0,4.212,4.212
1,2019-01-01 00:15:00,0.000,0.0,4.212,4.212
2,2019-01-01 00:30:00,0.000,0.0,4.212,4.212
3,2019-01-01 00:45:00,0.000,0.0,4.220,4.220
4,2019-01-01 01:00:00,0.000,0.0,4.212,4.212
5,2019-01-01 01:15:00,0.000,0.0,4.212,4.212
6,2019-01-01 01:30:00,0.000,0.0,4.212,4.212
7,2019-01-01 01:45:00,0.000,0.0,4.212,4.212
8,2019-01-01 02:00:00,0.000,0.0,4.220,4.220
9,2019-01-01 02:15:00,0.000,0.0,4.212,4.212
